# IATI Publisher Data Getter


| Version | Date | Description |
|-|-|-|
| 1.0 | 2019-09-23 | Initial run, investigating document link and related activity schema validation errors |
| 1.1 | 2019-09-23 | Updating the ad-hoc analysis section for clarity, expanding ruleset validation presentation |
| 2.0 | 2019-10-07 | Added non-current filter following PWYF rules, added codelist outputs to the ruleset evaluation, and added an exported .xslx file for ruleset evaluation sheets. |
| 2.1 | 2019-11-27 | (Ben W) Use lxml's huge_tree param to support bigger files |
|3.0| 2019-01-17 | (Ben W) Add file upload option, run CoVE validation on current activities only, split current_dict into a function so we can test it in another notebook, and run some checks from the IATI Publishing Statistics.
|||

Copyright (C) 2019 Open Data Services Co-operative Limited

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.


## Imports, Downloading, Merging


### Imports

In [89]:
! pip install XlsxWriter

In [0]:
import pandas as pd
import requests as rq
import lxml.etree as ET
import json
import shutil
import copy

### Setup

In [0]:
# Warning huge_tree disables security restrictions, so should only be used on trusted XML files
parser = ET.XMLParser(huge_tree=True)

#define comment removal method
def remove_comments(etree):
  
  comments = etree.xpath('//comment()')

  for c in comments:
      p = c.getparent()
      p.remove(c)

  return etree

### Download or Upload

In [0]:
data_mode = "Download" #@param ["Download", "Upload"]

#### Download

Input the IATI Registry ID of a given IATI Publisher. 

Set `current_only` to filter the combined IATI to consider only activities which are 'current' according to the [PWYF ATI Methodology](https://github.com/pwyf/latest-index-indicator-definitions/issues/1).

Set `current_until` to provide a date which will be used to calculate the current period (12 months prior to this date).

Exceptions can be added as an array of dataset_id strings such as this:

```json
exceptions: ["dataset1", "dataset2, "..."]
```

Currently only Activity files are supported.

In [0]:
if data_mode == "Download":
  registry_id = "dipr" #@param {type:"string"}
  filetype = "Activities" #@param ["Activities", "Organisations"]
  current_only = True #@param {type:"boolean"}
  exceptions = []
  current_until = "2020-01-29" #@param {type:"date"}
  exceptions =  []#@param {type:"raw"}

In [0]:
if data_mode == "Download":
  datasets = pd.read_csv("https://iatiregistry.org/csv/download/"+registry_id)

In [95]:
if data_mode == "Download":
  # remove unwanted datasets

  if filetype == "Activities":
    datasets = datasets[datasets['file-type'] != 'organisation']
  else: raise Exception('Currently, this notebook only supports IATI Activities, though could be easily modified to support Organisations')

  datasets = datasets[~datasets['registry-file-id'].isin(exceptions)]
  datasets = datasets.reset_index()

  print("Removed unwanted activities and setup comment-removal method")

Removed unwanted activities and setup comment-removal method


In [96]:
if data_mode == "Download":
  print("\nCombining {} IATI files \n".format(len(datasets['source-url'])))

  # Start with the first file, with comments removed
  big_iati = remove_comments(ET.fromstring(rq.get(datasets['source-url'][0]).content, parser=parser))

  # Start a dictionary to keep track of the additions
  merge_log = {datasets['source-url'][0]: len(big_iati.getchildren())}

  # Iterate through the 2nd through last file and
  # insert their activtities to into the first
  # and update the dictionary
  for url in datasets['source-url'][1:]:
      data = remove_comments(ET.fromstring(rq.get(url).content))
      merge_log[url] = len(data.getchildren())
      big_iati.extend(data.getchildren())

  # Print a small report on the merging
  print("Files Merged: ")
  for file, activity_count in merge_log.items():
      print("|-> {} activities from {}".format(activity_count, file))
  print("|--> {} in total".format(len(big_iati.getchildren())))

  with open("combined.xml", "wb+") as out_file:
      out_file.write(ET.tostring(big_iati, encoding='utf8', pretty_print=True))


Combining 2 IATI files 

Files Merged: 
|-> 7 activities from https://aidstream.s3.us-west-2.amazonaws.com/xml/dipr-activities.xml
|-> 78 activities from http://devinit.org/wp-content/uploads/2019/09/DIPR-IATI-data-Sep-2019.xml
|--> 85 in total


#### Upload

In [0]:
if data_mode == "Upload":
  filename = "upload.xml" #@param {type:"string"}
  current_only = True #@param {type:"boolean"}
  if current_only == True:
    current_until = "2020-01-29" #@param {type:"date"}

Go to View->"Table of contents" if the left pane isn't open, click Files at the top, then click Upload. Upload the file, then update `filename` below to match the filename you uploaded. Note that uploaded files will be cleared when the session ends.

In [0]:
if data_mode == "Upload":
  shutil.copyfile(filename, 'combined.xml')
  big_iati = remove_comments(ET.parse('combined.xml', parser=parser).getroot())

### Filter current activities

In [0]:
import datetime as dt
from dateutil.relativedelta import relativedelta

current_date = dt.datetime.strptime(current_until, "%Y-%m-%d")

def current_dict(activity):
  status_check = False
  planned_end_date_check = False
  actual_end_date_check = False
  transaction_date_check = False

  # print("Activity {} of {}".format(count, len(big_iati)))
  
  if activity.xpath("activity-status[@code=2]"):
    status_check = True

  if activity.xpath("activity-date[@type=3]/@iso-date"):
    date_time_obj = dt.datetime.strptime(activity.xpath("activity-date[@type=3]/@iso-date")[0], '%Y-%m-%d')
    if date_time_obj > (current_date - relativedelta(years=1)):
      planned_end_date_check = True
  
  if activity.xpath("activity-date[@type=4]/@iso-date"):
    date_time_obj = dt.datetime.strptime(activity.xpath("activity-date[@type=4]/@iso-date")[0], '%Y-%m-%d')
    if date_time_obj > (current_date - relativedelta(years=1)):
      actual_end_date_check = True

  if activity.xpath("transaction/transaction-type[@code=2 or @code=3 or @code=4]"):
    dates = activity.xpath("transaction[transaction-type[@code=2 or @code=3 or @code=4]]/transaction-date/@iso-date")
    date_truths = [dt.datetime.strptime(date, '%Y-%m-%d') > (current_date - relativedelta(years=1)) for date in dates]
    if True in date_truths:
      transaction_date_check = True

  pwyf_current = status_check or planned_end_date_check or actual_end_date_check or transaction_date_check

  return {
    'iati-identifier': activity.findtext('iati-identifier'),
    'status_check': status_check, 
    'planned_end_date_check': planned_end_date_check, 
    'actual_end_date_check': actual_end_date_check, 
    'transaction_date_check': transaction_date_check,
    'pwyf_current': pwyf_current,
  }
  

In [0]:
# Filter out non-current activities, if appropriate
# See https://github.com/pwyf/latest-index-indicator-definitions/issues/1

log_columns = ["iati-identifier", "status_check", "planned_end_date_check", "actual_end_date_check", "transaction_date_check", "pwyf_current"]
count = 1
current_check_log = pd.DataFrame(columns=log_columns)

for activity in big_iati:
  current_check_log = current_check_log.append(current_dict(activity), ignore_index=True)
  
  count = count + 1
    
current_check_log.to_csv("current_check_log.csv")

In [0]:
big_iati_archived = copy.copy(big_iati)

In [102]:
cur_length = len(big_iati)

if current_only:
  for activity in big_iati:
    if activity.findtext('iati-identifier') in current_check_log.loc[current_check_log['pwyf_current'] == False, 'iati-identifier'].values:
      activity.getparent().remove(activity)
  
  print("Removed {} non-current activities from a total of {}.".format((cur_length-len(big_iati)),cur_length))
  print("{} current activities remain.".format(len(big_iati)))

else:
  print("As `current_only` is set to False, all retrieved activities have been kept")

Removed 51 non-current activities from a total of 85.
34 current activities remain.


In [0]:
with open("combined_current.xml", "wb+") as out_file:
    out_file.write(ET.tostring(big_iati, encoding='utf8', pretty_print=True))

## Ad Hoc Analysis

This section can be used to evaluate specifica aspects of the total corpus of data, for instance, using `coverage_check()` you can look at the number of activities which include specific elements, or which satisfy certain contditions. This requires some python and XML knowledge. Some examples have been included below.

In [0]:
def coverage_check(tree, path, manual_list_entry=False):
  if manual_list_entry:
    denominator = len(tree)
    numerator = len(path)
  else:
    denominator = len(tree.getchildren())
    numerator = len(tree.xpath(path))

  coverage = numerator / denominator
  return denominator, numerator, coverage

In [105]:
coverage_check(big_iati, "iati-activity[transaction]")

(34, 34, 1.0)

In [106]:
coverage_check(big_iati, "iati-activity[capital-spend]")

(34, 0, 0.0)

In [107]:
# activities with a disbursement
coverage_check(big_iati, "iati-activity[transaction/transaction-type/@code = 3]")

(34, 0, 0.0)

In [108]:
# Manual entry of two lists to see the proportion of transactions which are disbursements
coverage_check(
    big_iati.xpath("iati-activity/transaction"), 
    big_iati.xpath("iati-activity/transaction[transaction-type/@code = 3]"), 
    True)

(2138, 0, 0.0)

## Batch CoVE Validation

In [109]:
json_validation_filepath = 'validation.json'

url = 'https://iati.cove.opendataservices.coop/api_test'
files = {'file': open("combined_current.xml", 'rb')}
r = rq.post(url, files=files, data={"name": "combined_current.xml"})

print(r)

print("CoVE validation was successful." if r.ok else "Something went wrong.")

validation_json = r.json()

with open(json_validation_filepath, "w") as out_file:
    json.dump(validation_json, out_file)

print('Validation JSON file has been written to {}.'.format(
    json_validation_filepath))

<Response [200]>
CoVE validation was successful.
Validation JSON file has been written to validation.json.


In [110]:
ruleset_table = pd.DataFrame(data=validation_json['ruleset_errors'])
schema_table = pd.DataFrame(data=validation_json['validation_errors'])
embedded_codelist_table = pd.DataFrame(data=validation_json['invalid_embedded_codelist_values'])
non_embedded_codelist_table = pd.DataFrame(data=validation_json['invalid_non_embedded_codelist_values'])

print(
    "CoVE has found: \n* {} schema errors \n* {} ruleset errors \n* {} embedded codelist errors \n* {} non-embedded codelist errors".format(
    len(schema_table), 
    len(ruleset_table), 
    len(embedded_codelist_table), 
    len(non_embedded_codelist_table)))

print("\nWriting to validation_workbook.xlsx")
writer = pd.ExcelWriter('validation_workbook.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
schema_table.to_excel(writer, sheet_name='schema_table')
ruleset_table.to_excel(writer, sheet_name='ruleset_table')
embedded_codelist_table.to_excel(writer, sheet_name='embedded_codelist_table')
non_embedded_codelist_table.to_excel(writer, sheet_name='non_embedded_codelist_table')

# Close the Pandas Excel writer and output the Excel file.
writer.save()


CoVE has found: 
* 0 schema errors 
* 1 ruleset errors 
* 2 embedded codelist errors 
* 6 non-embedded codelist errors

Writing to validation_workbook.xlsx


### Schema Validation

In [111]:
schema_table

""


#### Custom Analysis

This section gives space to investigate schema errors identified in the secion above. It requires a small amount of tinkering in python.

In [0]:
# To view offending XML element, take the '/NN/' from the path above, add one, 
# and then modify the remaining content of the path to print a section of XML.
# print(ET.tostring(big_iati.xpath("iati-activity[61]/related-activity")[0].getparent()).decode())

Note the lack of description in the XML output above

### Ruleset Validation

In [113]:
# Concise Summary
ruleset_table.pivot_table(index=['rule',], aggfunc='count').drop(columns=["path", "ruleset", "explanation"])

,id
rule,
activity-date[@type='4']/@iso-date must be today or in the past,1


In [114]:
# Full Table
ruleset_table.head()

,id,path,ruleset,explanation,rule
0,GB-COH-06368740-P0314,/iati-activities/iati-activity[20]/activity-da...,Actual dates must be in the past,2022-06-30 must be on or before today (2020-01...,activity-date[@type='4']/@iso-date must be tod...


## ATI Data Quality Testing

Download `combined.xml` and upload to [this testing tool](http://dataqualitytester.publishwhatyoufund.org/) to check data quality in line with the Aid Transparency Index Methodology

| Date | Link | Notes |
|-|-|-|
|YYYY-MM-DD|[Link](http://dataqualitytester.publishwhatyoufund.org/package/bb957674-6ccf-4635-a553-1d2dd0382075)| Some description of findings or link to notes / report |
||||
||||
||||
||||

## IATI Publisher Statistics

Run some code from the [OpenDataServices/iati-publishingstats-details](https://github.com/OpenDataServices/iati-publishingstats-details) repo, which produces per activity CSVs for the [IATI Publishing Statistics](http://publishingstats.iatistandard.org/) checks.

In [115]:
import os
os.chdir('/content')
!rm -r iati-publishingstats-details
!git clone https://github.com/OpenDataServices/iati-publishingstats-details.git
os.chdir('/content/iati-publishingstats-details')
!git submodule init
!git submodule update
!mkdir logs
# Note that the Publishing Statistics code requires Python 2, so needs a virtualenv
!sudo apt install python-virtualenv > logs/apt.log
!virtualenv .ve
!source .ve/bin/activate; pip install -r requirements.txt > logs/requirements.log
!source .ve/bin/activate; ./fetch_helpers.sh > logs/fetch_helpers.log 2>&1
!source .ve/bin/activate; python forward_looking_details.py ../combined.xml > forward_looking_details.csv
!source .ve/bin/activate; python comprehensiveness_is_current_details.py ../combined.xml > comprehensiveness_is_current_details.csv
os.chdir('/content')

Cloning into 'iati-publishingstats-details'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 30 (delta 12), reused 24 (delta 6), pack-reused 0
Unpacking objects: 100% (30/30), done.
Submodule 'IATI-Publishing-Statistics' (https://github.com/IATI/IATI-Publishing-Statistics.git) registered for path 'IATI-Publishing-Statistics'
Cloning into '/content/iati-publishingstats-details/IATI-Publishing-Statistics'...
Submodule path 'IATI-Publishing-Statistics': checked out '623498143ad809a57b87d357ea0dd0f65afd11b0'


Running virtualenv with interpreter /usr/bin/python2
New python executable in /content/iati-publishingstats-details/.ve/bin/python2
Also creating executable in /content/iati-publishingstats-details/.ve/bin/python
Installing setuptools, pkg_resources, pip, wheel...done.


In [116]:
os.chdir('/content')
forward_looking_details = pd.read_csv('iati-publishingstats-details/forward_looking_details.csv')
forward_looking_details

,iati-identifier,First year to fail,End dates,Budget years
0,GB-COH-06368740-P0258,2020,2020-12-31,2018
1,GB-COH-06368740-P0203,2020,2020-12-31,"2017,2017,2017,2017"
2,GB-COH-06368740-P0204,2020,2020-12-31,"2017,2017,2017,2017"
3,GB-COH-06368740-P0314,2020,2022-06-30,2018
4,GB-COH-06368740-P0341,2020,2022-05-31,No budgets
5,GB-COH-06368740-P0344,2020,2021-12-31,No budgets
6,GB-COH-06368740-P0358,2020,2021-12-31,No budgets


In [117]:
comprehensiveness_is_current_details = pd.read_csv('iati-publishingstats-details/comprehensiveness_is_current_details.csv')
comprehensiveness_is_current_details

,iati-identifier,publishingstats_comprehensiveness_current
0,GB-COH-06368740-P0255,False
1,GB-COH-06368740-P0283,False
2,GB-COH-06368740-P0256,False
3,GB-COH-06368740-P0144,False
4,GB-COH-06368740-P0044,True
...,...,...
80,GB-COH-06368740-P0353,False
81,GB-COH-06368740-P0356,False
82,GB-COH-06368740-P0357,False
83,GB-COH-06368740-P0358,True


In [118]:
big_iati_comprehensiveness_current = copy.copy(big_iati_archived)

cur_length = len(big_iati_comprehensiveness_current)

if current_only:
  for activity in big_iati_comprehensiveness_current:
    if activity.findtext('iati-identifier') in comprehensiveness_is_current_details.loc[comprehensiveness_is_current_details['publishingstats_comprehensiveness_current'] == False, 'iati-identifier'].values:
      activity.getparent().remove(activity)
  
  print("Removed {} non-current activities from a total of {}.".format((cur_length-len(big_iati_comprehensiveness_current)),cur_length))
  print("{} current activities remain.".format(len(big_iati_comprehensiveness_current)))

else:
  print("As `current_only` is set to False, all retrieved activities have been kept")

Removed 75 non-current activities from a total of 85.
10 current activities remain.


In [119]:
merged_currents = pd.merge(current_check_log, comprehensiveness_is_current_details, on="iati-identifier")
merged_currents.groupby(['pwyf_current', 'publishingstats_comprehensiveness_current']).size().unstack()
# Note: these numbers will not add up properly if there are duplicate iati-identifiers

publishingstats_comprehensiveness_current,False,True
pwyf_current,,
False,50,1
True,25,11
